In [1]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from safetensors.torch import load_file
from transformers import AutoTokenizer
import json
import math
from accelerate import init_empty_weights

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

shards = ['model-00001-of-00004.safetensors', 'model-00002-of-00004.safetensors', 'model-00003-of-00004.safetensors',
          'model-00004-of-00004.safetensors']

base_path = "/users/melodi/gsantoss/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e1945c40cd546c78e41f1151f4db032b271faeaa/"

state_dict = {}
for shard in shards:
    state_dict.update(load_file(base_path + shard))

with open(base_path + "config.json") as f:
    config = json.load(f)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
config['mlp_bias'] = False

In [4]:
for k, v in config.items():
    print(k, v)

architectures ['LlamaForCausalLM']
attention_bias False
attention_dropout 0.0
bos_token_id 128000
eos_token_id 128009
hidden_act silu
hidden_size 4096
initializer_range 0.02
intermediate_size 14336
max_position_embeddings 8192
model_type llama
num_attention_heads 32
num_hidden_layers 32
num_key_value_heads 8
pretraining_tp 1
rms_norm_eps 1e-05
rope_scaling None
rope_theta 500000.0
tie_word_embeddings False
torch_dtype bfloat16
transformers_version 4.40.0.dev0
use_cache True
vocab_size 128256
mlp_bias False


In [5]:
sample_prompt = '''apple: fruit
orange: fruit
zucchini: vegetable
tomato:

Complete this list'''

messages = [
    # {"role": "system", "content": "You are an Ontology Alignment expert. You are able to align two ontologies by creating a file in EDOAL format containing the result alignments. You are able to produce complex alignments that are those involving multiple entities and relationships in a n:m cardinality. The user will provide you with two ontologies and you respond with the EDOAL file containing the alignments. You don't need to explain yourself. Just give as response the resulting file without saying anything."},
    {"role": "user", "content": sample_prompt},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
)

print(input_ids.shape)

torch.Size([1, 29])


In [6]:
class Cache:
    
    def __init__(self):
        pass

class RotaryEmbedding(nn.Module):
    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None, scaling_factor=1.0):
        super().__init__()
        self.scaling_factor = scaling_factor
        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.base = base
        inv_freq = 1.0 / (self.base ** (torch.arange(0, self.dim, 2, dtype=torch.int64).float().to(device) / self.dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)

        self.max_seq_len_cached = max_position_embeddings

    @torch.no_grad()
    def forward(self, x, position_ids):
        inv_freq_expanded = self.inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1)
        position_ids_expanded = position_ids[:, None, :].float()

        device_type = x.device.type
        device_type = device_type if isinstance(device_type, str) and device_type != "mps" else "cpu"
        with torch.autocast(device_type=device_type, enabled=False):
            freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
            emb = torch.cat((freqs, freqs), dim=-1)
            cos = emb.cos()
            sin = emb.sin()
        return cos.to(dtype=x.dtype), sin.to(dtype=x.dtype)

def rotate_half(x):
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)


class SdpaAttention(nn.Module):
    def __init__(self, layer_idx, config, torch_dtype=torch.float32):
        super(SdpaAttention, self).__init__()
        self.layer_idx = layer_idx
        self.hidden_size = config['hidden_size']
        self.num_heads = config['num_attention_heads']
        self.head_dim = self.hidden_size // self.num_heads
        self.num_key_value_heads = config['num_key_value_heads']
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads

    
        self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=config['attention_bias'],
                                dtype=torch_dtype)
        self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim,
                                bias=config['attention_bias'], dtype=torch_dtype)
        self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim,
                                bias=config['attention_bias'], dtype=torch_dtype)
        self.o_proj = nn.Linear(self.hidden_size, self.hidden_size, bias=config['attention_bias'], dtype=torch_dtype)
        self.max_position_embeddings = config['max_position_embeddings']
        self.rope_theta = config['rope_theta']
        self.rotary_emb = RotaryEmbedding(self.head_dim, max_position_embeddings=self.max_position_embeddings, base=self.rope_theta)
        self.attention_dropout = config['attention_dropout']

    def forward(self, x, position_ids, kv_cache=None):
        bsz, q_len, _ = x.size()

        query_states = self.q_proj(x)
        key_states = self.k_proj(x)
        value_states = self.v_proj(x)
        
        query_states = query_states.view(bsz, -1, self.num_heads, self.head_dim).transpose(1, 2)
        key_states = key_states.view(bsz, -1, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        value_states = value_states.view(bsz, -1, self.num_key_value_heads, self.head_dim).transpose(1, 2)

        cos, sin = self.rotary_emb(value_states, position_ids)
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)
        
        if kv_cache is not None:
            if kv_cache[self.layer_idx] is not None:
                past_key, past_value = kv_cache[self.layer_idx]
                key_states = torch.cat([past_key, key_states], dim=2)
                value_states = torch.cat([past_value, value_states], dim=2)
            kv_cache[self.layer_idx] = (key_states, value_states)
        

        key_states = repeat_kv(key_states, self.num_key_value_groups)
        value_states = repeat_kv(value_states, self.num_key_value_groups)
        
        attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)

        causal_mask = torch.triu(torch.full((q_len, q_len), -1e9), diagonal=1)
        attn_weights = attn_weights + causal_mask

        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
        attn_weights = nn.functional.dropout(attn_weights, p=self.attention_dropout, training=self.training)
        attn_output = torch.matmul(attn_weights, value_states)

        attn_output = attn_output.transpose(1, 2).contiguous()
        attn_output = attn_output.view(bsz, q_len, -1)

        attn_output = self.o_proj(attn_output)

        return attn_output


class LLamaMLP(nn.Module):
    def __init__(self, config, torch_dtype=torch.float32):
        super(LLamaMLP, self).__init__()
        self.hidden_size = config['hidden_size']
        self.intermediate_size = config['intermediate_size']

        self.gate_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=config['mlp_bias'], dtype=torch_dtype)
        self.up_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=config['mlp_bias'], dtype=torch_dtype)
        self.down_proj = nn.Linear(self.intermediate_size, self.hidden_size, bias=config['mlp_bias'], dtype=torch_dtype)

        self.act_fn = nn.SiLU()

    def forward(self, x):
        return self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))


class RMSNorm(nn.Module):
    def __init__(self, hidden_size, torch_dtype, eps=1e-6):
        super(RMSNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size, dtype=torch_dtype))
        self.variance_epsilon = eps

    def forward(self, x):
        input_dtype = x.dtype
        hidden_states = x.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)


class DecoderLayer(nn.Module):
    def __init__(self, layer_idx, config, torch_dtype=torch.float32):
        super(DecoderLayer, self).__init__()
        self.layer_idx = layer_idx
        self.input_layernorm = RMSNorm(config['hidden_size'], torch_dtype, eps=config['rms_norm_eps'])

        self.self_attn = SdpaAttention(layer_idx, config, torch_dtype=torch_dtype)

        self.post_attention_layernorm = RMSNorm(config['hidden_size'], torch_dtype, eps=config['rms_norm_eps'])
        self.mlp = LLamaMLP(config, torch_dtype)

    def forward(self, x, position_ids, kv_cache=None):
        residual = x
        hidden_states = self.input_layernorm(x)
        attention_output = self.self_attn(hidden_states, position_ids, kv_cache=kv_cache)
        hidden_states = residual + attention_output

        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        hidden_states = residual + self.mlp(hidden_states)

        return hidden_states


class LLama(nn.Module):
    def __init__(self, config, torch_dtype=torch.float32):
        super(LLama, self).__init__()
        self.padding_idx = config['eos_token_id']

        self.embed_tokens = nn.Embedding(config['vocab_size'], config['hidden_size'], padding_idx=self.padding_idx,
                                         dtype=torch_dtype)
        
        self.layers = nn.ModuleList([DecoderLayer(layer_idx, config, torch_dtype) for layer_idx in range(config['num_hidden_layers'])])

        self.norm = RMSNorm(config['hidden_size'], torch_dtype, eps=config['rms_norm_eps'])

    def forward(self, x, position_ids=None, kv_cache=None):
        
        hidden_states = self.embed_tokens(x)

        for layer in self.layers:
            hidden_states = layer(hidden_states, position_ids, kv_cache=kv_cache)

        hidden_states = self.norm(hidden_states)

        return hidden_states


class LLamaGenerator(nn.Module):
    def __init__(self, config, torch_dtype=torch.float32):
        super(LLamaGenerator, self).__init__()
        self.config = config
        self.model = LLama(config, torch_dtype=torch_dtype)
        self.lm_head = nn.Linear(config['hidden_size'], config['vocab_size'], dtype=torch_dtype, bias=False)

    def forward(self, x, max_length=10, stop_token=None):
        inp = x
        fo = []
        kv_cache = [None] * self.config['num_hidden_layers']
        position_ids = torch.arange(inp.size(1)).unsqueeze(0).expand(x.size(0), -1)
        with torch.no_grad():
            for _ in tqdm(range(max_length)):
                out = self.model(inp, position_ids=position_ids, kv_cache=kv_cache)
                out = self.lm_head(out)[:, -1, :]
                out = out.argmax(-1).unsqueeze(-1)
                fo.append(out.item())
                if stop_token is not None and out.item() == stop_token:
                    break
                inp = out
                position_ids = torch.unsqueeze(position_ids[:, -1] + 1, 0)

        return fo

with init_empty_weights():
    generator = LLamaGenerator(config, torch_dtype=torch.bfloat16)
    
generator.load_state_dict(state_dict, assign=True)
generator.eval()



LLamaGenerator(
  (model): LLama(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128009)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): RMSNorm()
        (self_attn): SdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (post_attention_layernorm): RMSNorm()
        (mlp): LLamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_featu

In [7]:
out = generator(input_ids)

print(out)
tokenizer.decode(out)

  0%|          | 0/10 [00:00<?, ?it/s]

[32, 11670, 24790, 10368, 2268, 38501, 4428, 25, 14098, 128009]


'A classic classification exercise!\n\ntomato: fruit<|eot_id|>'